In [26]:
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

#not required in my case ...n_samples = 30
n_features = 50
n_components = 4 #five topics
n_top_words = 20 #ten words in each topic

In [27]:
%time
# dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=("headers", "footers", "quotes"))
# datasamples = dataset.data[:n_samples]


## own data from the amazon reviews

import pandas as pd

# location of the data
datafile = "../data/digitalmusic/reviews/complete_reviews.csv"

data = pd.read_csv(datafile, dtype={"asin": str, "helpful": str, "reviewText": str, "reviewerID": str})

# data.head

Wall time: 0 ns


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
## moulding the data to fit the LDA. 
# we will not find all the reviews for the item specific B00LOMZDM8

item_id1 = "B00LOMZDM8"

# for rows
row_data = data.loc[data['asin'] == item_id1]
# for columns
review_data = row_data.iloc[:,3]
## found 15 reviews for the item specific B00LOMZDM8

# datatext = ""
datatext = review_data.tolist()

#ratings on the same product
rating_data = row_data.iloc[:,2]

rating_data = rating_data.tolist()


# for review in data:
#     datatext += review+". "

print(datatext)

print(rating_data)
data.count()

['Catchy little tune', 'Awesome song', 'This is an amazing song', "I love the Yogscast team and I love there music. If your a fan you won't be let down!", 'Diggy Diggy Hole... nuff said.', "The Yogscast has done it again- they've somehow managed to turn a silly, lighthearted song into something truly epic. The song has come a long way since the early SOI!", 'I grew a beard and shrunk down to 3.8ft just by listening to this10/10 no regrets.', 'I just love this song, it has a happy beat, and my kids are easier to get going marching to this tune!', 'Perfect music for Dwarf Fortress', 'We really like this song/tune!  It has a great beat and fun to listen to :)', 'I love the yogscast and this song has got to be there best song yet!!', 'Definitely would dig again', "It's awesome", 'I AM DAVE YOGNAUT AND I HAVE THE BALLS!', "The Yogscast have done it again.  Another amazing song that I already can't get enough of.  Happy dance!"]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


asin          836007
helpful       836007
overall       836007
reviewText    835953
reviewerID    836005
summary       835981
dtype: int64

In [29]:
# tfidf vectorizer 

%time

# max_features is the key for the matrix vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(datatext)

Wall time: 0 ns


In [30]:
# tf vectorizer used for LDA - topic Modelling

%time
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(datatext)

Wall time: 0 ns


In [31]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=25, learning_method='online', learning_offset=50, random_state=0)

lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=25, mean_change_tol=0.001,
             n_components=4, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [32]:
# using visualisation for the lda
import pyLDAvis
import pyLDAvis.sklearn

tree = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer) # there are different variations of this function.

pyLDAvis.display(tree)



In [17]:

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [18]:
tf_feature_names = tf_vectorizer.get_feature_names()

print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: tune beat song amazing music awesome love yogscast just happy
Topic #1: amazing song just music happy yogscast awesome tune love beat
Topic #2: song awesome yogscast love amazing just tune beat happy music
Topic #3: song yogscast tune awesome amazing happy beat just music love
Topic #4: love tune music happy yogscast beat just song amazing awesome

